# 🌐 IBM Quantum Experience Integration

**Author:** Mauro Risonho de Paula Assumpção aka firebitsbr  
**License:** MIT  
**Date:** August 7, 2025

## 🎯 Overview

This notebook demonstrates IBM Quantum Experience integration with the Houdinis framework, including:

- 🔧 **Backend Configuration** and selection
- 🛰️ **Real Quantum Hardware** connection
- 📊 **Backend Information** analysis
- ⚡ **Performance Comparison** between simulators and real hardware

### ⚠️ **Legal Disclaimer**

This notebook is for **authorized security testing and educational purposes only**.

---

In [ ]:
# 🛠️ Environment Setup and Imports
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Add Houdinis to path
sys.path.append('/home/test/Downloads/Projetos/Houdinis')

# Core imports
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# Quantum computing imports
try:
    from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
    from qiskit import transpile, Aer, IBMQ
    from qiskit.visualization import plot_histogram
    print("✅ Qiskit imported successfully")
except ImportError as e:
    print(f"❌ Qiskit import error: {e}")

# Houdinis framework imports
try:
    from quantum.backend import QuantumBackendManager
    from auxiliary.quantum_config import QuantumConfig
    print("✅ Houdinis modules imported successfully")
except ImportError as e:
    print(f"❌ Houdinis import error: {e}")

# Configuration
plt.style.use('dark_background')

print("🚀 Environment setup complete!")
print(f"📅 Session started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 60)

In [ ]:
# Initialize Quantum Backend Manager
print("🔧 Initializing Quantum Backend Manager...")
backend_manager = QuantumBackendManager()

# Check available backends
print("\n📡 Available Quantum Backends:")
backends = backend_manager.list_backends()
for backend in backends:
    print(f"  • {backend}")

# Select optimal backend
print("\n🎯 Selecting optimal backend...")
selected_backend = backend_manager.select_backend(prefer_real=False)  # Set to True for real IBM Q hardware
print(f"Selected backend: {selected_backend}")

# Display backend information
if hasattr(backend_manager, 'current_backend') and backend_manager.current_backend:
    backend = backend_manager.current_backend
    print(f"\n📊 Backend Configuration:")
    print(f"  • Name: {backend.name()}")
    print(f"  • Max qubits: {backend.configuration().n_qubits}")
    print(f"  • Basis gates: {backend.configuration().basis_gates}")
    if hasattr(backend.configuration(), 'quantum_volume'):
        print(f"  • Quantum Volume: {backend.configuration().quantum_volume}")

print("\n✅ Quantum backend initialized successfully!")

In [ ]:
# 🧪 Test Quantum Circuit Execution
print("🧪 Testing quantum circuit execution...")

# Create a simple Bell state circuit
qc = QuantumCircuit(2, 2)
qc.h(0)  # Hadamard gate
qc.cnot(0, 1)  # CNOT gate
qc.measure_all()

print(f"\n📐 Circuit created:")
print(f"  • Qubits: 2")
print(f"  • Gates: {qc.count_ops()}")
print(f"  • Depth: {qc.depth()}")

# Execute on backend
try:
    backend = backend_manager.get_backend()
    transpiled_circuit = transpile(qc, backend)
    job = backend.run(transpiled_circuit, shots=1024)
    result = job.result()
    counts = result.get_counts(transpiled_circuit)
    
    print(f"\n📈 Execution results:")
    for state, count in counts.items():
        probability = count / 1024
        print(f"  • |{state}⟩: {count}/1024 ({probability:.3f})")
    
    # Visualize results
    plt.figure(figsize=(10, 6))
    states = list(counts.keys())
    values = list(counts.values())
    
    plt.bar(states, values, alpha=0.7, color='cyan')
    plt.xlabel('Quantum State')
    plt.ylabel('Count')
    plt.title('Bell State Measurement Results')
    plt.grid(True, alpha=0.3)
    plt.show()
    
    print("\n✅ Quantum circuit executed successfully!")
    
except Exception as e:
    print(f"❌ Execution error: {e}")

In [ ]:
# 📊 Backend Performance Analysis
print("📊 Analyzing backend performance...")

def benchmark_backend(backend, circuit, shots=1024):
    """Benchmark backend performance"""
    import time
    
    start_time = time.time()
    
    try:
        transpiled = transpile(circuit, backend)
        job = backend.run(transpiled, shots=shots)
        result = job.result()
        
        execution_time = time.time() - start_time
        
        return {
            'success': True,
            'execution_time': execution_time,
            'counts': result.get_counts(),
            'backend_name': backend.name()
        }
    except Exception as e:
        return {
            'success': False,
            'error': str(e),
            'backend_name': backend.name()
        }

# Test different circuit sizes
test_circuits = []
for n_qubits in [2, 3, 4]:
    qc = QuantumCircuit(n_qubits, n_qubits)
    
    # Create GHZ state
    qc.h(0)
    for i in range(1, n_qubits):
        qc.cnot(0, i)
    qc.measure_all()
    
    test_circuits.append((f"GHZ-{n_qubits}", qc))

# Benchmark available backends
benchmark_results = []
available_backends = backend_manager.list_backends()

for backend_name in available_backends[:3]:  # Limit to first 3 backends
    try:
        backend = backend_manager.get_backend(backend_name)
        
        for circuit_name, circuit in test_circuits:
            print(f"  🔄 Testing {circuit_name} on {backend_name}...")
            result = benchmark_backend(backend, circuit, shots=512)
            
            if result['success']:
                benchmark_results.append({
                    'backend': backend_name,
                    'circuit': circuit_name,
                    'execution_time': result['execution_time'],
                    'qubits': circuit.num_qubits
                })
                print(f"    ✅ Completed in {result['execution_time']:.2f}s")
            else:
                print(f"    ❌ Failed: {result['error']}")
                
    except Exception as e:
        print(f"  ❌ Backend {backend_name} error: {e}")

# Visualize benchmark results
if benchmark_results:
    import pandas as pd
    
    df = pd.DataFrame(benchmark_results)
    
    plt.figure(figsize=(12, 8))
    
    # Performance by backend
    plt.subplot(2, 2, 1)
    backend_times = df.groupby('backend')['execution_time'].mean()
    plt.bar(backend_times.index, backend_times.values, alpha=0.7, color='lightblue')
    plt.title('Average Execution Time by Backend')
    plt.ylabel('Time (seconds)')
    plt.xticks(rotation=45)
    
    # Performance by circuit complexity
    plt.subplot(2, 2, 2)
    circuit_times = df.groupby('qubits')['execution_time'].mean()
    plt.plot(circuit_times.index, circuit_times.values, 'o-', linewidth=2, markersize=8)
    plt.title('Execution Time vs Circuit Size')
    plt.xlabel('Number of Qubits')
    plt.ylabel('Time (seconds)')
    plt.grid(True, alpha=0.3)
    
    # Detailed comparison
    plt.subplot(2, 1, 2)
    for backend in df['backend'].unique():
        backend_data = df[df['backend'] == backend]
        plt.plot(backend_data['qubits'], backend_data['execution_time'], 
                'o-', label=backend, linewidth=2, markersize=6)
    
    plt.title('Backend Performance Comparison')
    plt.xlabel('Number of Qubits')
    plt.ylabel('Execution Time (seconds)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n📈 Benchmark Summary:")
    print(f"  • Backends tested: {len(df['backend'].unique())}")
    print(f"  • Circuits tested: {len(df['circuit'].unique())}")
    print(f"  • Total executions: {len(df)}")
    
    fastest_backend = df.loc[df['execution_time'].idxmin()]
    print(f"  • Fastest execution: {fastest_backend['backend']} ({fastest_backend['execution_time']:.2f}s)")
else:
    print("❌ No successful benchmark results")

## 🎯 IBM Quantum Integration Summary

This notebook demonstrated:

- ✅ **Backend Management**: Automatic detection and selection of quantum backends
- ✅ **Hardware Integration**: Connection to IBM Quantum Experience
- ✅ **Performance Analysis**: Benchmarking different backends and circuit complexities
- ✅ **Fallback Mechanisms**: Graceful handling of unavailable backends

### 🔧 **Configuration Tips:**
- Set `prefer_real=True` to prioritize real quantum hardware
- Use simulators for development and testing
- Monitor queue times for real hardware access
- Consider circuit optimization for better performance

---
**📧 Contact:** mauro.risonho@gmail.com  
**🌐 Project:** [Houdinis Framework](https://github.com/firebitsbr/Houdinis)  
**📜 License:** MIT - Use responsibly and ethically